In [ ]:
import kagglehub

path = kagglehub.dataset_download("mostafabakr8962/human-faces-dataset")

print("Path to dataset files:", path)

c:\Users\Mimo\OneDrive\Desktop\Project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Mimo\.cache\kagglehub\datasets\mostafabakr8962\human-faces-dataset\versions\1


In [ ]:
from __future__ import annotations


from pathlib import Path
from PIL import ImageFile
import torch
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, write_png

In [ ]:
# Custom Packages
from Packages.generator import Generator
from Packages.discriminator import Discriminator
from Packages.Imgprocessing import GrayscaleImageDataset

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
"""
from torchvision import transforms

transform1 = transforms.Compose([
    SoftFaceMaskTransform(mtcnn=mtcnn),
    transforms.Resize(280),                           
    transforms.CenterCrop(256),                      
    transforms.ToTensor(),                             
])

for f in os.listdir('Faces_Dataset'):
    if f.lower().endswith(('.jpg', '.png')):
        img_path = os.path.join('Faces_Dataset', f)
        
        try:
            img = Image.open(img_path).convert('RGB')
            img_gray = transform1(img)  
            
            h, w = img_gray.shape[1:]
            y, x = 128, 128
            y, x = min(y, h - 1), min(x, w - 1)
            
            should_delete = (img_gray[0, y, x] == 0)
            
            img.close() 

            if should_delete:
                print(f"Permanently deleting: {f}")
                os.remove(img_path)

        except Exception as e:
            print(f"Could not process {f}: {e}")
"""

'\ntransform1 = transforms.Compose([\n    SoftFaceMaskTransform(mtcnn=mtcnn),\n    transforms.Resize(280),                           \n    transforms.CenterCrop(256),                      \n    transforms.ToTensor(),                             \n])\n\nfor f in os.listdir(\'Faces_Dataset\'):\n    if f.lower().endswith((\'.jpg\', \'.png\')):\n        img_path = os.path.join(\'Faces_Dataset\', f)\n        \n        try:\n            img = Image.open(img_path).convert(\'RGB\')\n            img_gray = transform1(img)  \n            \n            h, w = img_gray.shape[1:]\n            y, x = 128, 128\n            y, x = min(y, h - 1), min(x, w - 1)\n            \n            should_delete = (img_gray[0, y, x] == 0)\n            \n            img.close() \n\n            if should_delete:\n                print(f"Permanently deleting: {f}")\n                os.remove(img_path)\n\n        except Exception as e:\n            print(f"Could not process {f}: {e}")\n'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

Using device: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
DATA_DIR = path + "/Faces_Dataset"


batch_size = 256
NUM_WORKERS = 0 if torch.cuda.is_available() else 2

print("Dataset dir:", DATA_DIR)
print("NUM_WORKERS:", NUM_WORKERS)

Dataset dir: C:\Users\Mimo\.cache\kagglehub\datasets\mostafabakr8962\human-faces-dataset\versions\1/Faces_Dataset
Run dir: runs\gan_faces
NUM_WORKERS: 0


In [ ]:
dataset = GrayscaleImageDataset(DATA_DIR)
if len(dataset) == 0:
    raise RuntimeError(f"No images found under {DATA_DIR!r}. Put your real face images there (jpg/png/...).")

pin_memory = device.type == "cuda"
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=NUM_WORKERS,
    pin_memory=pin_memory,
    persistent_workers=(NUM_WORKERS > 0),
)

batch = next(iter(dataloader))
print("Number of images:", len(dataset))
print("Batch shape:", batch.shape, "dtype:", batch.dtype, "min/max:", float(batch.min()), float(batch.max()))

C:\Users\Mimo\AppData\Local\Temp\ipykernel_15780\56460657.py:115: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  data = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes()))


Number of images: 124848
Batch shape: torch.Size([256, 1, 256, 256]) dtype: torch.float32 min/max: -2.222222328186035 2.222222328186035


In [ ]:
n_epochs = 50         
lr = 0.0002             
b1 = 0.5                
b2 = 0.999              
n_cpu = 4                

latent_dim = 128         
img_size = 256           
channels = 1             

sample_interval = 5    
img_shape = (channels, img_size, img_size)

In [ ]:
adversarial_loss = torch.nn.BCELoss()
cuda = True if torch.cuda.is_available() else False
generator = Generator(img_shape)
discriminator = Discriminator(img_shape)

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

In [ ]:
samples_dir = Path(r"C:\Users\Mimo\OneDrive\Desktop\Project\NN_proj\GANs-Generating-Human-Faces\samples")
samples_dir.mkdir(parents=True, exist_ok=True)

for epoch in range(n_epochs):
    for i, imgs in enumerate(dataloader):

        batch_size = imgs.size(0)

        valid = torch.ones(batch_size, 1, device=device)
        fake  = torch.zeros(batch_size, 1, device=device)

        real_imgs = imgs.to(device) 


        optimizer_G.zero_grad()

        z = torch.randn(batch_size, latent_dim, device=device)

        gen_imgs = generator(z) 

        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

 
        optimizer_D.zero_grad()

        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)

        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            f"[Epoch {epoch}/{n_epochs}] "
            f"[Batch {i}/{len(dataloader)}] "
            f"[D loss: {d_loss.item():.4f}] "
            f"[G loss: {g_loss.item():.4f}]"
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            grid = make_grid(gen_imgs[:25], nrow=5, normalize=True)
            grid_u8 = grid.mul(255).add_(0.5).clamp_(0, 255).to(torch.uint8).cpu()
            write_png(grid_u8, str(samples_dir / f"{batches_done}.png"))

[Epoch 0/50] [Batch 0/487] [D loss: 0.6758] [G loss: 0.7074]
[Epoch 0/50] [Batch 1/487] [D loss: 0.7581] [G loss: 0.6821]
[Epoch 0/50] [Batch 2/487] [D loss: 0.4155] [G loss: 0.6541]
[Epoch 0/50] [Batch 3/487] [D loss: 0.3802] [G loss: 0.6406]
[Epoch 0/50] [Batch 4/487] [D loss: 0.3763] [G loss: 0.6473]
[Epoch 0/50] [Batch 5/487] [D loss: 0.3649] [G loss: 0.6781]
[Epoch 0/50] [Batch 6/487] [D loss: 0.3310] [G loss: 0.7340]
[Epoch 0/50] [Batch 7/487] [D loss: 0.2967] [G loss: 0.8129]
[Epoch 0/50] [Batch 8/487] [D loss: 0.2707] [G loss: 0.9095]
[Epoch 0/50] [Batch 9/487] [D loss: 0.2647] [G loss: 0.9473]
[Epoch 0/50] [Batch 10/487] [D loss: 0.3355] [G loss: 0.7202]
[Epoch 0/50] [Batch 11/487] [D loss: 0.3149] [G loss: 0.7926]
[Epoch 0/50] [Batch 12/487] [D loss: 0.5488] [G loss: 0.8932]
[Epoch 0/50] [Batch 13/487] [D loss: 0.7130] [G loss: 0.2814]
[Epoch 0/50] [Batch 14/487] [D loss: 0.5994] [G loss: 0.3626]
[Epoch 0/50] [Batch 15/487] [D loss: 0.4269] [G loss: 0.5557]
[Epoch 0/50] [Batc

c:\Users\Mimo\OneDrive\Desktop\Project\.venv\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[Epoch 0/50] [Batch 118/487] [D loss: 0.5221] [G loss: 0.4658]
[Epoch 0/50] [Batch 119/487] [D loss: 0.4804] [G loss: 0.5039]
[Epoch 0/50] [Batch 120/487] [D loss: 0.4862] [G loss: 0.8010]
[Epoch 0/50] [Batch 121/487] [D loss: 0.6107] [G loss: 0.3793]
[Epoch 0/50] [Batch 122/487] [D loss: 0.4726] [G loss: 0.5020]
[Epoch 0/50] [Batch 123/487] [D loss: 0.2771] [G loss: 0.9090]
[Epoch 0/50] [Batch 124/487] [D loss: 0.4090] [G loss: 1.3886]
[Epoch 0/50] [Batch 125/487] [D loss: 0.3786] [G loss: 0.7465]
[Epoch 0/50] [Batch 126/487] [D loss: 0.3314] [G loss: 0.8061]
[Epoch 0/50] [Batch 127/487] [D loss: 0.2417] [G loss: 1.0315]
[Epoch 0/50] [Batch 128/487] [D loss: 0.2162] [G loss: 1.2305]
[Epoch 0/50] [Batch 129/487] [D loss: 0.2333] [G loss: 1.1498]
[Epoch 0/50] [Batch 130/487] [D loss: 0.2956] [G loss: 0.8311]
[Epoch 0/50] [Batch 131/487] [D loss: 0.3380] [G loss: 0.7486]
[Epoch 0/50] [Batch 132/487] [D loss: 0.4013] [G loss: 0.7467]
[Epoch 0/50] [Batch 133/487] [D loss: 0.4904] [G loss: 